<a href="https://colab.research.google.com/github/9-coding/Computer_Vision/blob/main/TA/baseline_VGG11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import time

import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image

In [3]:
### GPU Setting ###
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

from google.colab import drive
drive.mount('/content/gdrive/')

os.chdir('/content/gdrive/MyDrive/Colab Notebooks/Computer_Vision/ActiveLearning/CUB_200_2011_repackage_class50')
print(os.getcwd())

cuda
Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks/Computer_Vision/ActiveLearning/CUB_200_2011_repackage_class50


In [4]:
### Custom Dataset ###
class CUB2011(Dataset):
    def __init__(self, transform, mode='train'):
        self.transform = transform
        self.mode = mode

        if self.mode == 'train':
            self.image_folder = os.listdir('./datasets/train')
        elif self.mode == 'valid':
            self.image_folder = os.listdir('./datasets/valid')
        elif self.mode == 'test':
            self.image_folder = os.listdir('./datasets/test')

    def __len__(self):
        return len(self.image_folder)

    def __getitem__(self, idx):
        img_path = self.image_folder[idx]
        img = Image.open(os.path.join('./datasets', self.mode, img_path)).convert('RGB')
        img = self.transform(img)

        label = img_path.split('_')[-1].split('.')[0]
        label = int(label)
        return (img, label)

In [5]:
### Data Preprocessing ###
transforms_train = transforms.Compose([transforms.Resize((448, 448)),
                                       transforms.ToTensor()])

transforms_valtest = transforms.Compose([transforms.Resize((448, 448)),
                                         transforms.ToTensor()])

BATCH_SIZE = 32
train_set = CUB2011(mode='train',
                    transform=transforms_train)
val_set = CUB2011(mode='valid',
                  transform=transforms_valtest)
test_set = CUB2011(mode='test',
                   transform=transforms_valtest)

print('Num of each dataset:', len(train_set), len(val_set), len(test_set))

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

print("Loaded dataloader")

Num of each dataset: 2360 296 298
Loaded dataloader


In [16]:
### Model / Optimizer ###

EPOCH = 30
lr = 0.1

# Use VGG11 model with pre-trained weights
model = models.vgg11(pretrained=True)

### Transfer Learning ###
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 50)
print(model)
model.to(DEVICE)

optimizer = optim.SGD(model.parameters(), lr=lr)

print("Created a learning model and optimizer using VGG11")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [17]:
### Train/Evaluation ###
def train(model, train_loader, optimizer, epoch):
    model.train()
    for i, (image, target) in enumerate(train_loader):
        image, target = image.to(DEVICE), target.to(DEVICE)
        output = model(image)
        optimizer.zero_grad()
        train_loss = F.cross_entropy(output, target).to(DEVICE)

        train_loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(
                f'Train Epoch : {epoch} [{i}/{len(train_loader)}]\tLoss: {train_loss.item():.6f}')

    return train_loss

In [18]:
def evaluate(model, val_loader):
    model.eval()
    eval_loss = 0
    correct = 0
    with torch.no_grad():
        for i, (image, target) in enumerate(val_loader):
            image, target = image.to(DEVICE), target.to(DEVICE)
            output = model(image)

            eval_loss += F.cross_entropy(output,
                                         target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    eval_loss /= len(val_loader.dataset)
    eval_accuracy = 100 * correct / len(val_loader.dataset)
    return eval_loss, eval_accuracy

In [19]:
### Main ###
start = time.time()
best = 0
for epoch in range(EPOCH):
    train_loss = train(model, train_loader, optimizer, epoch)
    val_loss, val_accuracy = evaluate(model, val_loader)

    # Save best model
    if val_accuracy > best:
        best = val_accuracy
        torch.save(model.state_dict(), "./best_model.pth")

    print(f'[{epoch}] Validation Loss : {val_loss:.4f}, Accuracy: {val_accuracy:.4f}%')

Train Epoch : 0 [0/74]	Loss: 3.994552
Train Epoch : 0 [10/74]	Loss: 3.889665
Train Epoch : 0 [20/74]	Loss: 3.931416
Train Epoch : 0 [30/74]	Loss: 3.933871
Train Epoch : 0 [40/74]	Loss: 3.892371
Train Epoch : 0 [50/74]	Loss: 3.909106
Train Epoch : 0 [60/74]	Loss: 3.861313
Train Epoch : 0 [70/74]	Loss: 3.884990
[0] Validation Loss : 3.8539, Accuracy: 3.7162%
Train Epoch : 1 [0/74]	Loss: 3.849057
Train Epoch : 1 [10/74]	Loss: 3.940177
Train Epoch : 1 [20/74]	Loss: 3.889762
Train Epoch : 1 [30/74]	Loss: 3.907452
Train Epoch : 1 [40/74]	Loss: 3.798785
Train Epoch : 1 [50/74]	Loss: 3.800873
Train Epoch : 1 [60/74]	Loss: 3.682163
Train Epoch : 1 [70/74]	Loss: 3.907063
[1] Validation Loss : 3.6166, Accuracy: 7.4324%
Train Epoch : 2 [0/74]	Loss: 3.618076
Train Epoch : 2 [10/74]	Loss: 3.724081
Train Epoch : 2 [20/74]	Loss: 3.198192
Train Epoch : 2 [30/74]	Loss: 3.304050
Train Epoch : 2 [40/74]	Loss: 3.354304
Train Epoch : 2 [50/74]	Loss: 2.914758
Train Epoch : 2 [60/74]	Loss: 2.985941
Train Epoc

KeyboardInterrupt: 

In [ ]:
# Test result
test_loss, test_accuracy = evaluate(model, test_loader)
print(f'[FINAL] Test Loss : {test_loss:.4f}, Accuracy: {test_accuracy:.4f}%')

end = time.time()
elasped_time = end - start


print("Best Accuracy: ", best)
print(f"Elasped Time: {int(elasped_time/3600)}h, {int(elasped_time/60)}m, {int(elasped_time%60)}s")
print(f"time: {int(elasped_time/3600)}h, {int(elasped_time/60)}m, {int(elasped_time%60)}s")